# Coin segmentation

<div style="background: lightblue; width: calc(100%-2em); font-size: 150%; color: lightgray; padding: 1em; margin: 1em; line-height: 1.3em;">
Stéfan van der Walt<br/>
<a href="mailto:stefanv@berkeley.edu">stefanv@berkeley.edu</a><br/>
<a href="https://twitter.com/stefanvdwalt">@stefanvdwalt</a> on Twitter<br/>
<br/>
You can find this notebook online at https://github.com/stefanv/teaching
</div>


## Using watershed and region properties

In [1]:
import skimage
skimage.__version__

'0.14dev'

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from skimage import data
coins = data.coins()

In [ ]:
plt.imshow(coins, cmap='gray');

In [ ]:
from skimage import filters

coins_denoised = filters.median(coins, selem=np.ones((5, 5)))

f, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 5))
ax0.imshow(coins)
ax1.imshow(coins_denoised);

In [ ]:
from skimage import feature
edges = skimage.feature.canny(coins, sigma=3)

plt.imshow(edges);

In [ ]:
from scipy.ndimage import distance_transform_edt
dt = distance_transform_edt(~edges)

plt.imshow(dt);

In [ ]:
local_max = feature.peak_local_max(dt, indices=False, min_distance=5)
plt.imshow(local_max, cmap='gray');

In [ ]:
peak_idx = feature.peak_local_max(dt, indices=True, min_distance=5)
peak_idx[:5]

In [ ]:
plt.plot(peak_idx[1], peak_idx[0], 'r.')
plt.imshow(dt);

In [ ]:
from skimage import measure

markers = measure.label(local_max)

In [ ]:
from skimage import morphology, segmentation

labels = morphology.watershed(-dt, markers)
plt.imshow(segmentation.mark_boundaries(coins, labels));

In [ ]:
from skimage import color
plt.imshow(color.label2rgb(labels, image=coins));

In [ ]:
plt.imshow(color.label2rgb(labels, image=coins, kind='avg'), cmap='gray');

In [ ]:
regions = measure.regionprops(labels, intensity_image=coins)

In [ ]:
region_means = [r.mean_intensity for r in regions]
plt.hist(region_means, bins=20);

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2)

region_means = np.array(region_means).reshape(-1, 1)
model.fit(np.array(region_means).reshape(-1, 1))
print(model.cluster_centers_)

In [ ]:
bg_fg_labels = model.predict(region_means)
bg_fg_labels

In [ ]:
classified_labels = labels.copy()
for bg_fg, region in zip(bg_fg_labels, regions):
    classified_labels[tuple(region.coords.T)] = bg_fg

In [ ]:
plt.imshow(color.label2rgb(classified_labels, image=coins));